# Access Keyvault secrets from a Managed Online Endpoint

In this example we create a Keyvault, set a secret, and then retrieve the secret from a Managed Online Endpoint using the endpoint's system-assigned managed identity. By using the managed identity, the need to pass secrets as well as any other credentials in the image or deployment is avoided.

## 1. Configure parameters, assets, and clients

### 1.1 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

In [7]:
subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "alwallace-rg"
workspace_name = "alwallace"

### 1.2 Set endpoint and keyvault details

In [8]:
import random
rand = random.randint(0,10000)

endpoint_name = f"endpt-moe-kv-{rand}"
keyvault_name = f"kv{rand}"

### 1.3 Set asset paths

In [9]:
import os
base_path = "../../../../../cli/endpoints/online/managed/keyvault"
code_path = os.path.join(base_path, "code")
conda_file_path = os.path.join(base_path, "env.yml")


### 1.4 Examine the scoring script
The scoring script uses a `ManagedIdentityCredential` to authenticate itself to the Keyvault via a `SecretClient` from the `azure-keyvault` package. No arguments are needed to instantiate the credential object when this code is executed in a deployment, because it reads the environment variables `MSI_SECRET` and `MSI_ENDPOINT` which are already present by default.

```python 
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
import os
import json

secret_client : SecretClient = None

def init():
    cred = ManagedIdentityCredential()
    kv_name = os.getenv("KV_NAME") 

    global secret_client
    secret_client = SecretClient(vault_url=f"https://{kv_name}.vault.azure.net", credential=cred)

def run(data): 
    data = json.loads(data)
    name = data["name"]
    secret = secret_client.get_secret(name=name)
    return secret.value
```

### 1.5 Create an MLClient instance

In [10]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
